<a href="https://colab.research.google.com/github/Lorxus/Tontine/blob/main/tontine-deathcount-markovsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# numerical simulation for modeling daily death counts
# premise: each player has prob = 1/days+2 chance to die today and any day is like any other

import random as rand
import math

PLAYERCOUNT = 7141  # number of players in tontine, living and dead
YEARSTART = 312
LIVINGCOUNT = 277
DEATHSTHISYEAR = YEARSTART - LIVINGCOUNT

def howmanygone(numdays: int, timespan: int) -> int:  # simple markov simulation predicting the evolution of the tontine population after numdays days for the next timespan days
    deathchance = 1/(numdays + 2)  # Laplace's Rule of Succession
    deathcount = 0
    currentlyalive = LIVINGCOUNT

    if numdays == 0:
        return 0
    for i in range(timespan):
        for j in range(currentlyalive):
            if rand.random() < deathchance:
                deathcount += 1
                currentlyalive -= 1

    return deathcount

def ensemble(numworlds: int, numdays: int, timespan: int) -> float:  # ensemble simulation - number of runs, number of days it's been, number of days out predicted
    runoutcomes = [-1] * numworlds
    totaldeaths = 0
    for i in range(numworlds):
        thisrundeaths = howmanygone(numdays, timespan)
        runoutcomes[i] = thisrundeaths
        totaldeaths += thisrundeaths
        if rand.random() < 1/(numworlds ** 0.75):
                print('Run', i, '; deaths this time:', thisrundeaths)

    avgdeaths = (totaldeaths/numworlds)
    print('Average deaths:', avgdeaths)
    return avgdeaths

def mean_and_sd(data):  # calculates the mean, sd, and 95% CI of a dataset
    mean = sum(data)/len(data)
    deviationlist = []
    for d in data:
        tempdeviation = (d - mean) ** 2
        deviationlist.append(tempdeviation)

    avgdeviation = sum(deviationlist)/len(deviationlist)
    sd = avgdeviation ** 0.5
    upper = mean + sd * 1.97
    lower = mean - sd * 1.97
    print(f'mean: {mean:.0f}, sd: {sd:.3f}, 0.95 CI: {lower:.2f} <= x <= {upper:.2f}')
    return [mean, sd]

def ensemblethisyear(numworlds: int, numdays: int, timespan: int) -> float:  # ensemble simulation for this year - number of runs, number of days it's been, number of days out predicted
    runoutcomes = [DEATHSTHISYEAR] * numworlds  # technically I think this one's las vegas instead of monte carlo, effectively? measure-zero that it never halts.
    totaldeaths = 0
    for i in range(numworlds):
        thisrundeaths = howmanygone(numdays, timespan)
        runoutcomes[i] += thisrundeaths
        totaldeaths += thisrundeaths
        if rand.random() < 1/(numworlds ** 0.75):
                print('Run', i, '; deaths this time:', thisrundeaths)

    #print(runoutcomes)
    avgdeaths = totaldeaths/numworlds + DEATHSTHISYEAR
    print('Average deaths:', avgdeaths)
    mean = mean_and_sd(runoutcomes)[0]
    print(f'Predicted population on day {numdays + timespan}: {LIVINGCOUNT - mean:.2f}')

    return avgdeaths

def whenpophits(numdays: int, pop: int) -> int:  # reverse simulation - given a target pop count, how long does it take to get there?
    deathchance = 1/(numdays + 2)  # Laplace's Rule of Succession
    deathcount = 0
    currentlyalive = LIVINGCOUNT
    targetdeathcount = -1
    if pop < currentlyalive:
        targetdeathcount = currentlyalive - pop
    elif pop == currentlyalive:
        print('Starting and projected ending population counts are identical! Guaranteed to be zero.')
        return 0
    elif pop < 0:
        print('The population will never be negative.')
        return -1
    daycount = 0

    while deathcount < targetdeathcount:
        daycount += 1

        for i in range(currentlyalive):
            if rand.random() < deathchance:
                deathcount += 1
                currentlyalive -= 1

        # if rand.random() < 1/(numworlds ** 0.75):
        #     print('day', daycount, ':', deathcount, 'deaths so far')

        deathchance = 1/(numdays + daycount + 2)

    return daycount

def ensemblewhenpophits(numworlds: int, numdays: int, pop: int) -> float:  # ensemble reverse simulation as previous - number of runs, number of days it's been, target pop
    runoutcomes = [-1] * numworlds
    totaldays = 0
    for i in range(numworlds):
        thisrundays = whenpophits(numdays, pop)
        runoutcomes[i] = thisrundays
        totaldays += thisrundays
        if rand.random() < 1/(numworlds ** 0.75):
                print('Run', i, '; days taken this time:', thisrundays)

    if len(runoutcomes) < 101:
        print(sorted(runoutcomes))

    avgdays = totaldays/numworlds
    print('Average days taken:', avgdays)
    mean_and_sd(runoutcomes)

    return avgdays


print('Pick a mode: predicting the next specific period of time, or how long until a specific population. \'time\' for time-based, and \'pop\' for population-based.')
mode = input()
while mode != 'time' and mode != 'pop':
    print('Try that again. \'time\' for time-based, and \'pop\' for population-based.')
    mode = input()

if mode == 'time':
    print('How many days has it been since the start?')
    dayslong = int(input())
    print('How many days out are we predicting?')
    ticktock = int(input())
    print('How many ensemble runs?')
    numworlds = int(input())
    averagedeaths = ensemblethisyear(numworlds, dayslong, ticktock)

if mode == 'pop':
    print('How many days has it been since the start?')
    dayslong = int(input())
    print('What is the target population count?')
    leftover = int(input())
    print('How many ensemble runs?')
    numworlds = int(input())
    takesuntil = ensemblewhenpophits(numworlds, dayslong, leftover)


Pick a mode: predicting the next specific period of time, or how long until a specific population. 'time' for time-based, and 'pop' for population-based.
pop
How many days has it been since the start?
817
What is the target population count?
1
How many ensemble runs?
1024
Run 145 ; days taken this time: 111583
Run 169 ; days taken this time: 140615
Run 431 ; days taken this time: 226740
Run 641 ; days taken this time: 171445
Run 805 ; days taken this time: 100926
Run 811 ; days taken this time: 45363
Run 813 ; days taken this time: 145859
Average days taken: 221625.91796875
mean: 221626, sd: 382444.488, 0.95 CI: -531789.72 <= x <= 975041.56
